In [1]:
import numpy as np
import scipy as sy
import matplotlib.pyplot as plt
import sys
import os
import ast
import json
import random
from collections import defaultdict
from sklearn.model_selection import train_test_split
from ipynb.fs.full.Random_Sample_Mapper import *

In [2]:
notebook_path = os.path.abspath("BPR_OPT_Binary_Model.ipynb")
users_items_file_path = os.path.join(os.path.dirname(notebook_path), "data/australian_users_items.json")
items_file_path = os.path.join(os.path.dirname(notebook_path), "data/items_meta_data.json")
users_meta_data_file_path = os.path.join(os.path.dirname(notebook_path), "data/users_meta_data.json")

In [3]:
users_items = []
with open(users_items_file_path, 'r') as data:
    for line in data:
        users_items.append(ast.literal_eval(line))

In [4]:
with open(items_file_path, 'r') as data:
    games_dict = json.load(data)

In [5]:
with open(users_meta_data_file_path, 'r') as file:
    users_meta_data = json.load(file)

In [6]:
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)
playtimesPerItem = defaultdict(dict)
playtimesPerUser = defaultdict(dict)
itemNames = defaultdict(str)

In [7]:
train_usersPerItem = defaultdict(set)
train_itemsPerUser = defaultdict(set)
test_usersPerItem = defaultdict(set)
test_itemsPerUser = defaultdict(set)

In [8]:
for game in games_dict:
    if 'owners' in games_dict[game]:
        usersPerItem[game] = set(games_dict[game]['owners'].keys())

In [9]:
for user in users_items:
    u_id = user['user_id']
    items = [item['item_id'] for item in user['items']]
    itemsPerUser[u_id] = items
    
    items_train, items_test = train_test_split(items, test_size = 0.85)
    train_itemsPerUser[u_id] = items_train
    test_itemsPerUser[u_id] = items_test

    playtimesPerUser[user['user_id']] = dict((item['item_id'], item) for item in user['items'])
    for item in user['items']:
        itemNames[item['item_id']] = item['item_name']
        playtimesPerItem[item['item_id']][user['user_id']] = item

In [10]:
for user in train_itemsPerUser:
    for item in train_itemsPerUser[user]:
        train_usersPerItem[item].add(user)
        
    for item in test_itemsPerUser[user]:
        test_usersPerItem[item].add(user)

In [12]:
train_nUsers = len(train_itemsPerUser)
train_nItems = len(train_usersPerItem)
train_users = list(train_itemsPerUser.keys())
train_items = list(train_usersPerItem.keys())

In [13]:
user_item_counts = dict((k, len(v)) for k, v in train_itemsPerUser.items())

In [17]:
datafile = 'sample_in.tsv'
mapout1 = 'sample_map1.tsv'
mapout2 = 'sample_map2.tsv'
outfile = 'sample_out.tsv'

f = open(datafile,'w')
for user, items in train_itemsPerUser.items():
    for item in items:
        print(default_formatter(user,item), file=f)
f.close()

In [18]:
# run two stages of mapreduce
mapper1 = Mapper(user_item_counts, oversampling=10)
mapreduce(datafile, mapout1, mapper=mapper1, reducer=reducer)
mapreduce(datafile, mapout2, mapper=indicator_mapper)  # map the data again
mapreduce([mapout1, mapout2], outfile, reducer=indicator_reducer)

KeyboardInterrupt: 

# Complete Latent Factor Model with Binary Classification

A simple non-biased latent factor model that is wrapped into a binary function (sigmoid function) as a base line model, using popularity as the item's sole feature

In [49]:
itemBiases = defaultdict(float)

For each user and item we now have a low dimensional descriptor (representing that user's preferences, and that item's properties), of dimension K.

In [50]:
userGamma = {}
itemGamma = {}

In [51]:
K = 2

In [52]:
for u in playtimesPerUser:
    userGamma[u] = [random.random() * 0.1 - 0.05 for k in range(K)]

In [53]:
for i in playtimesPerItem:
    itemGamma[i] = [random.random() * 0.1 - 0.05 for k in range(K)]

We'll use another library in this example to perform gradient descent. This library requires that we pass it a "flat" parameter vector (theta) containing all of our parameters. This utility function just converts between a flat feature vector, and our model parameters, i.e., it "unpacks" theta into our offset and bias parameters.

In [54]:
def unpack(theta):
    global itemBiases
    global userGamma
    global itemGamma
    index = 0
    itemBiases = dict(zip(train_users, theta[0:index + train_nItems]))
    index += train_nItems
    for u in train_users:
        userGamma[u] = theta[index:index + K]
        index += K
    for i in train_items:
        itemGamma[i] = theta[index:index + K]
        index += K

In [55]:
def inner(x, y):
    return sum([a*b for a,b in zip(x,y)])

## Sigmoid Function

\begin{equation*}
\sigma(x) = \frac{1}{1 + e^{-x}}
\end{equation*}

In [56]:
def sigmoid(x):
    #Numerically stable sigmoid function.
    #Taken from: https://timvieira.github.io/blog/post/2014/02/11/exp-normalize-trick/
    if x >= 0:
        z = np.exp(-x)
        return 1 / (1 + z)
    else:
        # if x is less than zero then z will be small, denom can't be
        # zero because it's 1+z.
        z = np.exp(x)
        return z / (1 + z)

## Prediction Function

\begin{equation*}
f(u, i, j) = \gamma_u \gamma_i + \beta_i - (\gamma_u \gamma_j + \beta_j)
\end{equation*}

\begin{equation*}
p(i >_u j) = \sigma(f(u, i, j))
\end{equation*}

In [57]:
def prediction(user, item_i, item_j):
    return inner(userGamma[user], itemGamma[item_i]) + itemBiases[item_i] - (inner(userGamma[user], itemGamma[item_j]) + itemBiases[item_j]) 

## Cost Function

\begin{equation*}
\text{BPR-OPT} := \text{argmax} \ln(\sigma(\gamma_u \gamma_i + \beta_i - (\gamma_u \gamma_j + \beta_j)))
\end{equation*}

\begin{equation*}
\text{Cost Function}:= \sum_{u,i,j} -ln\left( 1 + e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)} \right)
\end{equation*}

In [58]:
def cost(theta):
    unpack(theta)
    cost = 0
    for u, i, j in sample:
        x = prediction(u, i , j)
        cost += exp.log(sigmoid(x))
        
    return -cost

## Gradient Ascent

\begin{equation*}
\frac{\partial f}{\partial \gamma_{u,k}} = \frac{(\gamma_{i,k} - \gamma_{j,k}) \cdot e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}{1 + e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}
\end{equation*}

\begin{equation*}
\frac{\partial f}{\partial \gamma_{i,k}} = \frac{\gamma_{u,k} \cdot e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}{1 + e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}
\end{equation*}

\begin{equation*}
\frac{\partial f}{\partial \gamma_{j,k}} = -\frac{\gamma_{u,k} \cdot e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}{1 + e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}
\end{equation*}

\begin{equation*}
\frac{\partial f}{\partial \beta_i} = \frac{e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}{1 + e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}
\end{equation*}

\begin{equation*}
\frac{\partial f}{\partial \beta_j} = -\frac{e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}{1 + e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}
\end{equation*}

In [59]:
def derivative(theta):
    unpack(theta)
    dItemBiases = defaultdict(float)
    dUserGamma = {}
    dItemGamma = {}
    for u in train_users:
        dUserGamma[u] = [0.0 for k in range(K)]
    for i in train_items:
        dItemGamma[i] = [0.0 for k in range(K)]
    for u, i, j in sample:
        x = prediction(u, i ,j)
        dbase = np.exp(-x) / (1 + np.exp(-x))
        dItemBiases[i] += dbase
        dItemBiases[j] += -dbase
        for k in range(K):
            dUserGamma[u][k] += (itemGamma[i][k] - itemGamma[j][k]) * dbase
            dItemGamma_k = userGamma[u][k] * dbase
            dItemGamma[i][k] += dItemGamma_k
            dItemGamma[j][k] += -dItemGamma_k
    dtheta = [dItemBiases[i] for i in train_items]
    for u in train_users:
        dtheta += dUserGamma[u]
    for i in train_items:
        dtheta += dItemGamma[i]
    return numpy.arrray(dtheta)